In [1]:
import gzip
import pickle
import matplotlib.pyplot as plt
import yaml
import io
import os
import numpy as np
np.seterr(all='warn')
import hist
from typing import Any, IO, Dict, List
import dctools
from dctools import plot as plotter
import matplotlib.pyplot as plt
import scipy.interpolate as interp
from scipy import stats as st

In [6]:
from cycler import cycler

all_ch = set([
    'catSignal-0jet', 
])

def plotting(config, variable, channel, rebin=1, xlim=[], blind=False, era="someyear") -> None:
    datasets:Dict = dict()
    color_cycle:List = []
    for name in config.groups:
        histograms = dict(
            filter(
                lambda _n: _n[0] in config.groups[name].processes,
                config.boosthist.items()
            )
        )
        print(" --> name : ",name,  list(histograms.keys()) )
        p = dctools.datagroup(
            histograms = histograms,
            ptype      = config.groups[name].type,
            observable = variable,
            name       = name,
            xsections  = config.xsections,
            channel    = channel,
            luminosity = config.luminosity.value,
            rebin      = rebin
        )
        
        datasets[p.name] = p
        if p.ptype == "signal":
            signal = p.name
            
        if hasattr(config.groups[name], "color") and len(p.to_boost().shape):
            color_cycle.append(config.groups[name].color)
    
    _plot_channel = plotter.add_process_axis(datasets)
    pred = _plot_channel.project('process', 'systematic', variable)[:hist.loc('data'),:,:]   
    data = _plot_channel[{'systematic':'nominal'}].project('process', variable)[hist.loc('data'),:]
    

    plt.figure(figsize=(6,7))
    ax, bx = plotter.mcplot(
        pred[{'systematic':'nominal'}].stack('process'),
        data=None if blind else data,
        syst=pred.stack('process'),
        colors = color_cycle
    )
    
    ymax = np.max([10000]+[c.get_height() for c in ax.containers[0] if ~np.isnan(c.get_height())])
    ymin = np.min([0.001]+[c.get_height() for c in ax.containers[0] if ~np.isnan(c.get_height())])
    
    ax.set_ylim(0.001, 1000*ymax)
    
    try:
        sig_ewk = _plot_channel[{'systematic':'nominal'}].project('process', variable)[hist.loc('VBSZZ2l2nu'),:]   
        sig_qcd = _plot_channel[{'systematic':'nominal'}].project('process', variable)[hist.loc('ZZ2l2nu'),:]   
        sig_ewk.plot(ax=ax, histtype='step', color='red')
        sig_qcd.plot(ax=ax, histtype='step', color='purple')
    except:
        pass
    # bx.set_ylim([0.1, 1.9])
    if len(xlim) > 0:
        bx.set_xlim(xlim)
        
    ax.set_title(f"channel {channel}: {era}")
    ax.set_yscale('log')
    
    plt.savefig(f'plot-{channel}-{variable}-{era}.pdf')
    plt.savefig(f'plot-{channel}-{variable}-{era}.png')
    return _plot_channel, datasets

In [7]:
config_2016 = dctools.read_config("config/input_DAS_2016.yaml")

In [8]:
dir(config_2016)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cfg',
 'boosthist']

In [9]:
for channel in config_2016.plotting:
    ch_cfg = config_2016.plotting[channel]
    for vname in ch_cfg:
        print(ch_cfg, vname)
        v_cfg = ch_cfg[vname]
        plotting(
            config_2016, vname, channel, 
            rebin=v_cfg.rebin, 
            xlim=v_cfg.range, 
            blind=v_cfg.blind,
            era="2018"
        )

<dctools.config_input object at 0x7f3f3e4433d0> met_pt
 --> name :  WW ['WWTo2L2Nu_13TeV-powheg']
loop : cat3L WWTo2L2Nu_13TeV-powheg StrCategory(['catSignal-0jet', 'catEM', 'catSignal-1jet'], growth=True, name='channel')
 --> name :  WZ ['WZTo2L2Q_13TeV_amcatnloFXFX_madspin_pythia8', 'WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8']
loop : cat3L WZTo2L2Q_13TeV_amcatnloFXFX_madspin_pythia8 StrCategory(['catSignal-0jet', 'catEM', 'catSignal-1jet'], growth=True, name='channel')
loop : cat3L WZTo3LNu_TuneCUETP8M1_13TeV-powheg-pythia8 StrCategory(['catSignal-0jet', 'catEM', 'catSignal-1jet'], growth=True, name='channel')
 --> name :  Top ['ST_tW_antitop_5f_inclusiveDecays_13TeV-powheg-pythia8_TuneCUETP8M1', 'ST_tW_top_5f_inclusiveDecays_13TeV-powheg-pythia8_TuneCUETP8M1', 'TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8', 'TTTo2L2Nu_TuneCUETP8M2_ttHtranche3_13TeV-powheg-pythia8', 'TTWJetsToLNu_TuneCUETP8M1_13TeV-amcatnloFXFX-madspin-pythia8', 'TTWJetsToQQ_TuneCUETP8M1_13TeV-amcatnloFXFX-madsp

UnboundLocalError: local variable 'axes' referenced before assignment

In [ ]:
config_2016.boosthist